In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/aiscuser/.conda/envs/handbook_02_phi3_38/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "phi3_noacad_v0improved_3_3_8b_phase2_2024_03_22_hf"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
print(tokenizer)

LlamaTokenizerFast(name_or_path='phi3_noacad_v0improved_3_3_8b_phase2_2024_03_22_hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'additional_special_tokens': ['<|/inst|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=False),
	32000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<|step|>", rstrip=True, lstrip=

In [5]:
model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True, torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2")

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.44s/it]


In [6]:
print(model.dtype)

torch.bfloat16


In [7]:
# debug
print(tokenizer.eos_token)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token)
print(tokenizer.pad_token_id)
print(tokenizer.bos_token)
print(tokenizer.bos_token_id)

tokens = tokenizer.tokenize("Turing Models are awesome.")
print(tokens)

# generate
text = "translate welcome in Hindi"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

inputs = tokenizer(text, return_tensors="pt", return_attention_mask=True).to(DEVICE)
model = model.to(DEVICE)

with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)

output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output)

<|endoftext|>
32000
None
None
<s>
1
['▁T', 'uring', '▁Mod', 'els', '▁are', '▁awesome', '.']


/home/aiscuser/.conda/envs/handbook_02_phi3_38/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


NameError: name '_flash_supports_window_size' is not defined

In [ ]:
print(model)